Checking and understandig the data I am working with.

In [2]:
import pandas as pd
import numpy as np

URL='https://raw.githubusercontent.com/Patrick0481/Individual-project/refs/heads/main/activities.csv'
Strava_df = pd.read_csv(URL)

Strava_df.head()

,Activity ID,Activity Date,Activity Name,Activity Type,Activity Description,Elapsed Time,Distance,Max Heart Rate,Relative Effort,Commute,...,Intensity,Average Grade Adjusted Pace,Timer Time,Total Cycles,Recovery,With Pet,Competition,Long Run,For a Cause,Media
0,3072138967,"Feb 4, 2020, 7:06:50 PM",Evening Run,Run,NaN,2784,7.83,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3091650151,"Feb 11, 2020, 6:58:01 PM",Evening Run,Run,NaN,3396,9.44,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3111748506,"Feb 18, 2020, 7:05:14 PM",Evening Run,Run,NaN,2531,7.92,184.0,95.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3137405227,"Feb 27, 2020, 7:16:17 PM",Evening Run,Run,NaN,2998,8.05,185.0,109.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3151633969,"Mar 3, 2020, 7:16:03 PM",Evening Run,Run,NaN,2355,7.24,186.0,106.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
Strava_df.info()
Strava_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172 entries, 0 to 1171
Data columns (total 99 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Activity ID                   1172 non-null   int64  
 1   Activity Date                 1172 non-null   object 
 2   Activity Name                 1172 non-null   object 
 3   Activity Type                 1172 non-null   object 
 4   Activity Description          266 non-null    object 
 5   Elapsed Time                  1172 non-null   int64  
 6   Distance                      1172 non-null   float64
 7   Max Heart Rate                1091 non-null   float64
 8   Relative Effort               1091 non-null   float64
 9   Commute                       1172 non-null   bool   
 10  Activity Private Note         0 non-null      float64
 11  Activity Gear                 544 non-null    object 
 12  Filename                      1168 non-null   object 
 13  Ath

,Activity ID,Elapsed Time,Distance,Max Heart Rate,Relative Effort,Activity Private Note,Athlete Weight,Bike Weight,Elapsed Time.1,Moving Time,...,Training Load,Intensity,Average Grade Adjusted Pace,Timer Time,Total Cycles,Recovery,With Pet,Competition,Long Run,For a Cause
count,1.172000e+03,1172.000000,1172.000000,1091.000000,1091.000000,0.0,0.0,7.0,1172.000000,1172.000000,...,0.0,0.0,517.000000,0.0,0.0,116.0,0.0,0.0,0.0,0.0
mean,1.074758e+10,3168.318259,9.403797,172.624198,70.513291,NaN,NaN,9.0,3168.318259,2564.477816,...,NaN,NaN,3.483389,NaN,NaN,0.0,NaN,NaN,NaN,NaN
std,3.820100e+09,1989.868861,8.973242,15.976586,42.255954,NaN,NaN,0.0,1989.868861,1463.580150,...,NaN,NaN,0.453835,NaN,NaN,0.0,NaN,NaN,NaN,NaN
min,3.072139e+09,68.000000,0.000000,119.000000,1.000000,NaN,NaN,9.0,68.000000,68.000000,...,NaN,NaN,2.511000,NaN,NaN,0.0,NaN,NaN,NaN,NaN
25%,7.955700e+09,1850.250000,5.555000,160.000000,40.000000,NaN,NaN,9.0,1850.250000,1607.250000,...,NaN,NaN,3.227000,NaN,NaN,0.0,NaN,NaN,NaN,NaN
50%,1.127694e+10,2784.500000,8.185000,172.000000,66.000000,NaN,NaN,9.0,2784.500000,2493.000000,...,NaN,NaN,3.392000,NaN,NaN,0.0,NaN,NaN,NaN,NaN
75%,1.387678e+10,4191.500000,10.952500,181.000000,94.000000,NaN,NaN,9.0,4191.500000,3291.250000,...,NaN,NaN,3.593000,NaN,NaN,0.0,NaN,NaN,NaN,NaN
max,1.676829e+10,13760.000000,81.910000,234.000000,306.000000,NaN,NaN,9.0,13760.000000,10597.000000,...,NaN,NaN,5.772000,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [4]:
# Remove columns with all NaN values
print(f"Column count for cleanup: {len(Strava_df.columns)}")

# Clean the DataFrame by dropping columns with all NaN values
Strava_df = Strava_df.dropna(axis=1, how='all')

# Final overview
print(f"Columns after cleaning: {len(Strava_df.columns)}")
print(Strava_df.info())

Column count for cleanup: 99
Columns after cleaning: 64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172 entries, 0 to 1171
Data columns (total 64 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Activity ID                  1172 non-null   int64  
 1   Activity Date                1172 non-null   object 
 2   Activity Name                1172 non-null   object 
 3   Activity Type                1172 non-null   object 
 4   Activity Description         266 non-null    object 
 5   Elapsed Time                 1172 non-null   int64  
 6   Distance                     1172 non-null   float64
 7   Max Heart Rate               1091 non-null   float64
 8   Relative Effort              1091 non-null   float64
 9   Commute                      1172 non-null   bool   
 10  Activity Gear                544 non-null    object 
 11  Filename                     1168 non-null   object 
 12  Bike Weight         

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Filter alleen op hardlopen ('Run')
data = Strava_df[Strava_df['Activity Type'] == 'Run']

# Zet de datum om naar een getal (timestamp) zodat het model trends kan leren
data['Activity Date'] = pd.to_datetime(data['Activity Date'], format='mixed')
data['Date_Numeric'] = data['Activity Date'].map(pd.Timestamp.timestamp)

X = data[['Distance', 'Elevation Gain', 'Date_Numeric', 'Average Heart Rate', 'Relative Effort', 'Moving Time', 'Average Speed']].fillna(0)
y = data['Moving Time'] / 60.0 # Omzetten naar minuten

# 2. Splitsen in train- en testdata
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Data standaardiseren (belangrijk voor Keras)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Het Keras model bouwen (simpel feed-forward netwerk)
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[3]), # 3 inputs
    layers.Dense(32, activation='relu'),
    layers.Dense(1)                                       # 1 output (de tijd)
])

model.compile(optimizer='adam', loss='mae')

# 5. Model trainen
model.fit(X_train_scaled, y_train, epochs=50, verbose=0)

# 6. Voorspelling doen voor 5km vandaag
import time
huidige_datum = time.time()

# Input voor voorspelling: [Afstand, Hoogtemeters, Datum]
nieuwe_run = [[5.0, 0.0, huidige_datum]] 
nieuwe_run_scaled = scaler.transform(nieuwe_run)

voorspelde_tijd = model.predict(nieuwe_run_scaled)
print(f"Voorspelde tijd op 5km: {voorspelde_tijd[0][0]:.2f} minuten")

C:\Users\patri\AppData\Local\Temp\ipykernel_25164\3759935583.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Activity Date'] = pd.to_datetime(data['Activity Date'], format='mixed')
C:\Users\patri\AppData\Local\Temp\ipykernel_25164\3759935583.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date_Numeric'] = data['Activity Date'].map(pd.Timestamp.timestamp)
c:\Users\patri\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass 

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_6" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (None, 7)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 7), dtype=float32)
  • training=True
  • mask=None
  • kwargs=<class 'inspect._empty'>